In [1]:
import os
import json

from credentials import OPENAPI_KEY
os.environ["OPENAI_API_KEY"] = OPENAPI_KEY

import openai
from tqdm import tqdm
import pandas as pd

In [2]:
client = openai.OpenAI()

In [3]:
ROOT_PATH = "PATH"
ds_name = "6kTwitter"
ds_path = f"{ROOT_PATH}/dataGPT4o/{ds_name}.jsonl"
with open(ds_path, "r") as f:
    my_dataset = [json.loads(line) for line in f]

In [6]:
my_dataset[0]

{'messages': [{'role': 'system',
   'content': 'VRL-GPT-4o Türkçe metinlerin duygusunu verebilen bir chatbottur. (Pozitif, Nötr, Negatif)'},
  {'role': 'user',
   'content': 'Bu Türkçe metin hangi duyguyu içeriyor?: "düğümünde bir şartla altin istemem damat  medcezirin tum sezonunu 2 kere izlerse ?_?"? (Pozitif/Nötr/Negatif)'},
  {'role': 'assistant', 'content': 'Negatif'}]}

In [11]:
preds = []
refs  = []

for example in tqdm(my_dataset):
    response = client.chat.completions.create(
        model="gpt-4o",  # or "gpt-4-turbo"
        messages=example["messages"][:2]
    )
    
    model_output = response.choices[0].message.content
    ground_truth = example["messages"][-1]["content"]
    preds.append(model_output)
    refs.append(ground_truth)

100%|██████████| 6001/6001 [1:17:43<00:00,  1.29it/s] 


In [12]:
output_dir = f"{ROOT_PATH}/outputs"
output_df  = pd.DataFrame({"preds": preds, "refs": refs})
output_df.to_csv(f"{output_dir}/gpt4o-{ds_name}.csv", index=False)

In [13]:
output_df["preds"].value_counts()

preds
Negatif                                                                                                                                                                                                                                      1323
Negatif.                                                                                                                                                                                                                                     1163
Pozitif.                                                                                                                                                                                                                                     1014
Pozitif                                                                                                                                                                                                                                       712
Nötr                      